## Skipgram in PyTorch

### Good reads before going through the codes

-[Tutorial build your own embedding](https://blog.cambridgespark.com/tutorial-build-your-own-embedding-and-use-it-in-a-neural-network-e9cde4a81296)

-[Code](https://github.com/DSKSD/DeepNLP-models-Pytorch/blob/master/notebooks/02.Skip-gram-Negative-Sampling.ipynb)

-[Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)

## 1. Theory

![skipgram](https://cdn-images-1.medium.com/max/800/1*SR6l59udY05_bUICAjb6-w.png)


### (1) Softmax Objective
Skip-gram’s objective is to predict the contexts, given a target word: $V_t \rightarrow V_c$ 

The contexts are immediate neighbours of the target and are retrieved using a window of an arbitrary size _n_ 
    
Capturing _n_ words to the left of the target and _n_ words to its right.

In a two-gram example:

$$\underbrace{\textrm{The quick}}_{\textrm{left } n}\underbrace{\textrm{ brown }}_{target} \underbrace{\textrm{for jumps}}_{\textrm{right } n}$$

<img src="https://nbviewer.jupyter.org/github/DSKSD/DeepNLP-models-Pytorch/blob/master/images/01.skipgram-prepare-data.png">

The original Skip-gram's objective is to maximise $P(V_c|V_t)$: The probability of $V_c$ being predicted as $V_t$’s context for all training pairs.

![objective](https://miro.medium.com/max/700/1*aBBoTq_b_zQBwR2gi9mMkA.png)

To calculate $P(V_c|V_t)$ we need a way to quantify the __closeness__ of the target-word and the context-word. 

In Skip-gram, this closeness is computed using the __dot product between the input-embedding of the target and the output-embedding of the context__.

Now, if we define $u_{t,c}$ to be the measure of words' closeness between the target word and context word, $E$ to be the embedding matrix holding input-embeddings and $O$ to be the output-embedding matrix we get:

![dotproduct](https://miro.medium.com/max/372/1*z6pc-3WV7CGiJQq2Xca7bA.png)


which we can use to compute P(Vc|Vt) using the softmax function:

![softmax](https://miro.medium.com/max/2400/1*tWAXNe83b5Le9mKx0oeIpQ.png)



### (2) Architecture
In terms of the architecture, Skip-gram is a simple neural network with only one hidden layer. The input to the network is a one-hot encoded vector representation of a target-word — all of its dimensions are set to zero, apart from the dimension corresponding to the target-word. The output is the probability distribution over all words in the vocabulary which defines the likelihood of a word being selected as the input word’s context:

![archit1](https://miro.medium.com/max/2400/1*4Viy_LvP6jLIWSvB9-Fk-Q.png)

![archit2](https://miro.medium.com/max/456/1*q-BAjhE79g7_od8XuXC4Xg.png)


### (3) Negative Sampling

But there is an issue with the original softmax objective of Skip-gram — it is highly computationally expensive, as it requires scanning through the output-embeddings of all words in the vocabulary in order to calculate the sum from the denominator. And typically such vocabularies contain hundreds of thousands of words. Because of this inefficiency most implementations use an alternative, negative-sampling objective, which rephrases the problem as a set of independent binary classification tasks.

Instead of defining the complete probability distribution over words, the model learns to differentiate between the correct training pairs retrieved from the corpus and the incorrect, randomly generated pairs. For each correct pair the model draws m negative ones — with m being a hyperparameter. All negative samples have the same Vt as the original training pair, but their Vc is drawn from an arbitrary noise distribution. Building on the previous example, for the training pair (fox, jump) the incorrect ones could be (fox, man) or (fox, truck). The new objective of the model is to maximise the probability of the correct samples coming from the corpus and minimise the corpus probability for the negative samples, such as (fox, truck).

Let’s set D to be the set of all correct pairs and D’ to denote a set of all negatively sampled |D| × m pairs. We will also define P(C = 1|Vt, Vc) to be the probability of (Vt , Vc) being a correct pair, originating from the corpus. Given this setting, the negative-sampling objective is defined as maximising:

![negsamequ](https://miro.medium.com/max/2400/1*Uns2130EOoQY0AM9Lwf9fg.png)


Since this time for each sample we are making a binary decision we define P(C = 1|Vt, Vc) using the sigmoid function:
![nesamequ2](https://miro.medium.com/max/485/1*V5g7NuFg_cyYranubVRYcA.png)

where, as before, uc = Et · Oc. Now, if we plug this into the previous negative-sampling equation and simplify a little we get the following objective:

![negsamequ3](https://miro.medium.com/max/584/1*YZhOEMKyZGDEoAi8OOUDBA.png)

## 2. Import and Define some functions

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import nltk
import numpy as np
import random
from collections import Counter

import pdb

flatten = lambda l: [item for sublist in l for item in sublist]
random.seed(888)

In [94]:
## getBatch function return data in batches

def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        sindex = eindex
        eindex += batch_size
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [6]:
# prepare_sequence change sentance to list of index eg [5 124 421 0 3764 124 1 345]
# prepare_word change words to index eg 4523
# index type is longTensor (64-bit integer)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.Tensor(idxs).type(torch.LongTensor)

def prepare_word(word, word2index):
    return torch.Tensor([word2index[word]]).type(torch.LongTensor) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]) 


## 3. Data loading and Preprocessing

In [7]:
# download corpus
nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tanya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\tanya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

##### Gutenberg Corpus
NLTK includes a small selection of texts from the Project Gutenberg electronic text archive, which contains some 25,000 free electronic books, hosted at http://www.gutenberg.org/. We begin by getting the Python interpreter to load the NLTK package, then ask to see `nltk.corpus.gutenberg.fileids()`, the file identifiers in this corpus:

In [8]:
nltk.corpus.gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [11]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500] # sampling sentences for test
# lowercase
corpus = [[word.lower() for word in sent] for sent in corpus]

print(len(corpus))
print(corpus[0], len(corpus[0]))

500
['[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']'] 8


In [12]:
# Exclude sparse words
# word count more than 3 are remove

MIN_COUNT = 3
word_count = Counter(flatten(corpus))
exclude = []
for w, c in word_count.items():
    if c < MIN_COUNT:
        exclude.append(w)

In [16]:
print(len(exclude))

2129


In [18]:
# remove sparse words
vocab = list(set(flatten(corpus)) - set(exclude))
vocab.append('<UNK>')
print(vocab)

['english', 'sometimes', 'london', 'did', 'chapter', 'entry', 'find', 't', 'chace', 'stern', 'water', 'she', 'cape', 'who', 'another', 'beast', 'less', 'there', 'red', 'and', 'dim', '?--', 'sing', 'passage', 'again', 'great', 'altogether', 'dictionary', 'teeth', 'winds', 'bright', 'compare', 'than', 'paying', 'mind', 'place', 'without', 'form', 'me', 'enter', 'he', 'black', 'myself', 'though', 'that', 'go', 'against', 'when', 'quantity', 'd', 'parts', 'last', 'till', 'could', 'under', 'name', 'jaw', 'royal', 'here', 'may', 'said', 'each', 'same', 'especially', '!"', 'young', 'while', 'ibid', 'three', 'globe', 'summer', 'tail', 'whales', 'voyages', 'much', 'looking', 'eyes', 'sleeps', 'give', 'run', 'should', 'spermacetti', ',"', 'too', 'would', 'open', 'island', 'feet', 'sperma', 'think', 'blows', 'monsters', 'sign', 'ago', 'whale', 'poor', 'tempestuous', 'inn', 'immense', 'blue', 'from', 'city', 'entering', '."', 'hand', 'if', 'ceti', 'history', 'within', 'making', 'northern', 'part',

In [19]:
# create a word2index dictionary
# first key is '<UNK>'

word2index = {'<UNK': 0}

for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)

# create the opposite dictionary index2word

index2word = {v: k for k, v in word2index.items()}

In [34]:
# create the windows of word data to feed into the model
# window size mean the number of word before and after the center word
WINDOW_SIZE =5

# The dummy variable'<DUMMY>' is use to pad 
# ie if center word is the first word in the sentance need 5 dummy variable in front
windows = flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE,
                                    WINDOW_SIZE * 2 +1)) for c in corpus])
print(len(windows))

10277


In [33]:
# print preview first few windows
print(windows[0])
print(windows[1])
print(windows[2])
print(windows[3])
print(windows[4])
print(windows[5])
print(windows[6])
print(windows[7])
print()
print(windows[8])

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville')
('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851')
('<DUMMY>', '<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']')
('<DUMMY>', '<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>')
('<DUMMY>', '[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>')
('[', 'moby', 'dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>')
('moby', 'dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>')
('dick', 'by', 'herman', 'melville', '1851', ']', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>')

('<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>', 'etymology', '.', '<DUMMY>', '<DUMMY>', '<DUMMY>', '<DUMMY>')


In [37]:
# create training set
## run through the window in wondows
## find the all the context and target pairs
## words in exclude list and dummy variable will not be include

# window[WINDOW_SIZE]: target word
# window[i]          : context word

train_data = []
for window in windows:
    for i in range(len(window)):
        if window[i] in exclude or window[WINDOW_SIZE] in exclude:
            continue
        if window[i] == '<DUMMY>' or i == WINDOW_SIZE:
            continue
        
        train_data.append((window[WINDOW_SIZE], window[i]))

In [39]:
# print first few example

train_data[:20]

[('(', 'supplied'),
 ('(', 'by'),
 ('(', 'a'),
 ('(', 'late'),
 ('supplied', '('),
 ('supplied', 'by'),
 ('supplied', 'a'),
 ('supplied', 'late'),
 ('by', '('),
 ('by', 'supplied'),
 ('by', 'a'),
 ('by', 'late'),
 ('by', 'to'),
 ('a', '('),
 ('a', 'supplied'),
 ('a', 'by'),
 ('a', 'late'),
 ('a', 'to'),
 ('a', 'a'),
 ('late', '(')]

In [41]:
# convert train_data into index as postive data


X_p = []
y_p = []

for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1,-1))
    y_p.append(prepare_word(tr[1], word2index).view(1,-1))
    
train_data = list(zip(X_p, y_p))

In [42]:
# print first few example

train_data[:20]

[(tensor([[291]]), tensor([[186]])),
 (tensor([[291]]), tensor([[130]])),
 (tensor([[291]]), tensor([[215]])),
 (tensor([[291]]), tensor([[451]])),
 (tensor([[186]]), tensor([[291]])),
 (tensor([[186]]), tensor([[130]])),
 (tensor([[186]]), tensor([[215]])),
 (tensor([[186]]), tensor([[451]])),
 (tensor([[130]]), tensor([[291]])),
 (tensor([[130]]), tensor([[186]])),
 (tensor([[130]]), tensor([[215]])),
 (tensor([[130]]), tensor([[451]])),
 (tensor([[130]]), tensor([[207]])),
 (tensor([[215]]), tensor([[291]])),
 (tensor([[215]]), tensor([[186]])),
 (tensor([[215]]), tensor([[130]])),
 (tensor([[215]]), tensor([[451]])),
 (tensor([[215]]), tensor([[207]])),
 (tensor([[215]]), tensor([[215]])),
 (tensor([[451]]), tensor([[291]]))]

In [43]:
len(train_data)

50242

## 4. Unigram Distribution

Here, we need to create a distribution for negative sampling. Because in a corpus, some words appear more than other words. Hence it will not be fair randomly pick word from that distribution. we need tp create Unigram Distribution.


In [44]:
word_count= Counter(flatten(corpus))
num_total_words = sum([c for w, c in word_count.items() if w not in exclude])

In [51]:
alpha = 3/4
noise_dist = {key: val/num_total_words ** alpha for key, val in word_count.items() if key not in exclude}
Z = sum(noise_dist.values())

noise_dist_normalized = {key: val/Z for key, val in noise_dist.items()}

In [52]:
print("Normalization factor Z:        {:.7}".format(Z))

# noise distribution of the word 'by'
print('Noise distribution:            {:.6}'.format(noise_dist["by"]))
print('Normalized noise distribution: {:.6}'.format(noise_dist_normalized["by"]))

Normalization factor Z:        9.397142
Noise distribution:            0.0566383
Normalized noise distribution: 0.00602719


In [54]:
# randomly select 10 words
K=10

np.random.choice(list(noise_dist_normalized.keys()), size=K, p=list(noise_dist_normalized.values()), replace=True)

array([',', ';', '-', 'i', 'well', 'king', ',', 's', 'of', 'such'],
      dtype='<U11')

In [86]:
# Negative sampling function

def negative_sampling(targets, noise_dist_normalized, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        if device =='cuda':
            #GPU to CPU
            target_index = targets[i].data.cpu().tolist()[0] # PyTorch tensor to list
        else:
            target_index = targets[i].tolist()[0]
    
        while len(nsample) < k: #num of sampling
            neg = np.random.choice(list(noise_dist_normalized.keys()), size=1, p=list(noise_dist_normalized.values()))
            
            neg_word = neg[0]
            if word2index[neg_word] == target_index:
                continue
            nsample.append(neg_word)
        neg_samples.append(prepare_sequence(nsample, word2index).view(1,-1))
        
    return torch.cat(neg_samples)

In [95]:
class SkipgramNegSampling(nn.Module):
    
    def __init__(self, vocab_size, embed_dim):
        super(SkipgramNegSampling, self).__init__()  
        self.embedding_v = nn.Embedding(vocab_size, embed_dim) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, embed_dim) # out embedding
        self.logsigmoid = nn.LogSigmoid()

        nn.init.xavier_normal_(self.embedding_v.weight)
        nn.init.xavier_normal_(self.embedding_u.weight)
        
     
    def forward(self, center_words, target_words, negative_words):
        center_embeds = self.embedding_v(center_words) # B x 1 x D
        target_embeds = self.embedding_u(target_words) # B x 1 x D
        
        neg_embeds = -self.embedding_u(negative_words) # B x K x D

        positive_score = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2) # B x 1
        negative_score = torch.sum(neg_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2), 1).view(negs.size(0), -1) # B x K -> B x 1
        
        loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)

        return -torch.mean(loss)

    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds

## 5. Training

In [96]:
EMBEDDING_SIZE = 30 
BATCH_SIZE = 256
EPOCH = 30
NEG = 10 # Num of Negative Sampling

In [97]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device: {}".format(device))

loss = []
model = SkipgramNegSampling(len(word2index), EMBEDDING_SIZE)
if device == 'cuda':
    model = model.to(device)
    
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

Device: cpu


In [98]:
# show model

model

SkipgramNegSampling(
  (embedding_v): Embedding(480, 30)
  (embedding_u): Embedding(480, 30)
  (logsigmoid): LogSigmoid()
)

In [99]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [100]:
for epoch in range(EPOCH):
    scheduler.step()
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        inputs, targets =zip(*batch)
        
        inputs = torch.cat(inputs).to(device) # B x 1
        targets = torch.cat(targets).to(device) # B x 1
        negs = negative_sampling(targets, noise_dist_normalized, NEG).to(device)

        model.zero_grad()
        
        loss = model(inputs, targets, negs)
        
        loss.backward()
        optimizer.step()
        
        if i % 20 ==0:
            lr = get_lr(optimizer)
            print(f"Epoch : {epoch} || Iter: {i} || Loss : {loss:.2f} || LR: {lr:.6f}")        
        

Epoch : 0 || Iter: 0 || Loss : 1.39 || LR: 0.001000
Epoch : 0 || Iter: 20 || Loss : 1.37 || LR: 0.001000
Epoch : 0 || Iter: 40 || Loss : 1.34 || LR: 0.001000
Epoch : 0 || Iter: 60 || Loss : 1.25 || LR: 0.001000
Epoch : 0 || Iter: 80 || Loss : 1.12 || LR: 0.001000
Epoch : 0 || Iter: 100 || Loss : 1.04 || LR: 0.001000
Epoch : 0 || Iter: 120 || Loss : 0.98 || LR: 0.001000
Epoch : 0 || Iter: 140 || Loss : 0.94 || LR: 0.001000
Epoch : 0 || Iter: 160 || Loss : 0.93 || LR: 0.001000
Epoch : 0 || Iter: 180 || Loss : 0.92 || LR: 0.001000
Epoch : 1 || Iter: 0 || Loss : 0.91 || LR: 0.001000
Epoch : 1 || Iter: 20 || Loss : 0.91 || LR: 0.001000
Epoch : 1 || Iter: 40 || Loss : 0.91 || LR: 0.001000
Epoch : 1 || Iter: 60 || Loss : 0.90 || LR: 0.001000
Epoch : 1 || Iter: 80 || Loss : 0.91 || LR: 0.001000
Epoch : 1 || Iter: 100 || Loss : 0.90 || LR: 0.001000
Epoch : 1 || Iter: 120 || Loss : 0.90 || LR: 0.001000
Epoch : 1 || Iter: 140 || Loss : 0.90 || LR: 0.001000
Epoch : 1 || Iter: 160 || Loss : 0.90 ||

Epoch : 15 || Iter: 60 || Loss : 0.87 || LR: 0.000001
Epoch : 15 || Iter: 80 || Loss : 0.88 || LR: 0.000001
Epoch : 15 || Iter: 100 || Loss : 0.88 || LR: 0.000001
Epoch : 15 || Iter: 120 || Loss : 0.88 || LR: 0.000001
Epoch : 15 || Iter: 140 || Loss : 0.88 || LR: 0.000001
Epoch : 15 || Iter: 160 || Loss : 0.88 || LR: 0.000001
Epoch : 15 || Iter: 180 || Loss : 0.88 || LR: 0.000001
Epoch : 16 || Iter: 0 || Loss : 0.90 || LR: 0.000001
Epoch : 16 || Iter: 20 || Loss : 0.87 || LR: 0.000001
Epoch : 16 || Iter: 40 || Loss : 0.89 || LR: 0.000001
Epoch : 16 || Iter: 60 || Loss : 0.88 || LR: 0.000001
Epoch : 16 || Iter: 80 || Loss : 0.89 || LR: 0.000001
Epoch : 16 || Iter: 100 || Loss : 0.87 || LR: 0.000001
Epoch : 16 || Iter: 120 || Loss : 0.88 || LR: 0.000001
Epoch : 16 || Iter: 140 || Loss : 0.89 || LR: 0.000001
Epoch : 16 || Iter: 160 || Loss : 0.88 || LR: 0.000001
Epoch : 16 || Iter: 180 || Loss : 0.87 || LR: 0.000001
Epoch : 17 || Iter: 0 || Loss : 0.87 || LR: 0.000001
Epoch : 17 || Iter: 

## 6. Test

In [101]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index).to(device))

    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: 
            continue
            
        vector = model.prediction(prepare_word(list(vocab)[i], word2index).to(device))
        
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [102]:
test = random.choice(list(vocab))
print(test)
word_similarity(test, vocab)

sing


[['killed', 0.7773110270500183],
 ['yourself', 0.7749539017677307],
 ['young', 0.7704371213912964],
 ['within', 0.7560813426971436],
 ['thou', 0.7482627630233765],
 ['large', 0.7398788332939148],
 ['side', 0.7367793917655945],
 ['air', 0.7355208992958069],
 ['whaling', 0.7344357967376709],
 ['cook', 0.7241607308387756]]

In [103]:
word_similarity('time', vocab)

[['their', 0.8415759205818176],
 ['doubtless', 0.8235083222389221],
 ['gone', 0.8141038417816162],
 ['lord', 0.811817467212677],
 ['island', 0.8009188771247864],
 ['else', 0.7869023084640503],
 ['aloft', 0.7854390144348145],
 ['door', 0.7827306389808655],
 ['stream', 0.7791623473167419],
 ['looked', 0.7790824174880981]]

# Thank you!